<a href="https://colab.research.google.com/github/siva-sankar-a/erav1/blob/main/session_12_pytorch_lightning/Session_12_Pytorch_Lightning_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ERA V1 Session 12

## Refer [link](https://github.com/siva-sankar-a/erav1/tree/session/session-6-multiple-architecture-check/session_6_multiple_architectures_mnist#readme) for further documentation

## Cloning the Github repository
## [Session 12](https://github.com/siva-sankar-a/erav1.git)

In [45]:
!pip freeze > requirements.txt

In [1]:
!pip install torchviz
!pip install torch-lr-finder
!pip install grad-cam
!pip install torchmetrics
!pip install pytorch_lightning
!pip install gradio

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=ae6271d6de74dccddbd6fa50888403a5065185420785b2f23545126ec8eb07bd
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.8-py3-none-any.whl size=38245 sha256=92764b995ecb401c5ff3ee9f0b3b50569af049fb13db1dd8391ed985863d31c0
  Stored in directory: /root/.cache/pip/wheels/f8/04/36/94ff3c8a4215826a21946b34c01180817e606989fdf53f7cd6
Successfully built grad-cam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 11.1 MB/s eta 0:00:00
     ━

In [2]:
import numpy as np
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2

## Get device

In [29]:
import glob
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import pytorch_lightning as pl
import torchmetrics
import torch.optim as optim

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [15]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
print("CUDA Available?", cuda)

CUDA Available? True


In [16]:
labels = {0: 'airplane',
          1: 'automobile',
          2: 'bird',
          3: 'cat',
          4: 'deer',
          5: 'dog',
          6: 'frog',
          7: 'horse',
          8: 'ship',
          9: 'truck'}

In [17]:
pd.options.plotting.backend = "plotly"

In [18]:
class Net(nn.Module):

    '''
    This class implements the neural network model

    Usage:
    >>> device = ...
    >>> model = Net().to('cpu')
    '''

    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()

        # Prep Layer
        self.prep = self.conv3x3_bn_dropout(3, 64, padding=1)

        # Layer 1
        self.conv_bn_pool_1 = self.conv3x3_maxpool(64, 128)
        # Layer 1 - residual
        self.res_1 = self.conv3x3_bn_dropout(128, 128, padding=1)
        self.res_2 = self.conv3x3_bn_dropout(128, 128, padding=1)
        self.downsample_1 = self.conv1x1(128, 128)

        # Layer 2
        self.conv_bn_pool_2 = self.conv3x3_maxpool(128, 256)

        # Layer 3
        self.conv_bn_pool_3 = self.conv3x3_maxpool(256, 512)
        # Layer 3 - residual
        self.res_3 = self.conv3x3_bn_dropout(512, 512, padding=1)
        self.res_4 = self.conv3x3_bn_dropout(512, 512, padding=1)
        self.downsample_2 = self.conv1x1(512, 512)

        # Pool
        self.pool = nn.MaxPool2d(4)

        # Linear
        self.linear = nn.Linear(512, 10)

    def conv3x3_maxpool(self, in_channels, out_channels):
        layers = [self.conv3x3_bn_dropout(in_channels, out_channels, padding=1),
                  nn.MaxPool2d(2)]

        return nn.Sequential(
                    *layers
                )

    def conv3x3_bn_dropout(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=False, dropout=None, is_relu_bn=True):

          if is_relu_bn:
            layers = [
                  self.conv3x3(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias),
                  nn.BatchNorm2d(out_channels),
                  nn.ReLU(),
            ]
          else:
            layers = [
                  self.conv3x3(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
            ]

          if dropout:
            return nn.Sequential(
                    *layers,
                    nn.Dropout(dropout)
                )
          else:
            return nn.Sequential(
                    *layers
                )

    def conv3x3(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=False):
          return nn.Conv2d(in_channels,
                    out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    dilation=dilation,
                    groups=groups,
                    bias=bias)

    def conv1x1(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False):
          return nn.Conv2d(in_channels,
                    out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    bias=bias)


    def forward(self, x):

        # Prep Layer
        x = self.prep(x)

        # Layer 1
        x = self.conv_bn_pool_1(x)
        # Layer 1 - residual
        i = x
        x = self.res_1(x)
        x = self.res_2(x)
        x = x + i

        # Layer 2
        x = self.conv_bn_pool_2(x)

        # Layer 3
        x = self.conv_bn_pool_3(x)
        # Layer 3 - residual
        i = x
        x = self.res_3(x)
        x = self.res_4(x)
        x = x + i

        # Pool
        x = self.pool(x)

        # Flatten
        x = x.view(-1, 512)

        # Linear
        x = self.linear(x)

        return x

### Pytorch Lightning trainer

In [20]:
class NetPL(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = Net()
        self.num_classes = 10
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=self.num_classes)

    def forward(self, x):
        embedding = self.model(x)
        return embedding

    def configure_optimizers(self):

        num_epochs = 20
        momentum = 0.9
        max_lr = 1E-02
        regularization = None
        epochs_up = 7
        base_momentum = 0.85
        div_factor = 100

        steps_per_epoch = len(train_dataloader)
        total_steps = num_epochs * steps_per_epoch
        pct_start = 0.3

        optimizer = optim.Adam(self.model.parameters(), lr=0.1)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                                max_lr=max_lr,
                                                total_steps=total_steps,
                                                epochs=num_epochs,
                                                steps_per_epoch=steps_per_epoch,
                                                pct_start=pct_start,
                                                anneal_strategy='linear',
                                                cycle_momentum=True,
                                                base_momentum=base_momentum,
                                                max_momentum=momentum,
                                                div_factor=div_factor,
                                                verbose=False)

        return [optimizer], [{'scheduler': scheduler, 'interval': 'step'}]

    def training_step(self, train_batch, batch_idx):
        cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction='mean')
        data, target = train_batch

        output = self.model(data)

        logits = F.log_softmax(output, dim=1)
        pred = torch.argmax(logits, dim=1)

        self.accuracy(pred, target)
        loss = cross_entropy_loss(output, target)

        self.log('train_acc_step', self.accuracy, prog_bar=True, logger=True)
        self.log('train_loss', loss, prog_bar=True, logger=True)

        return loss

    def validation_step(self, val_batch, batch_idx):
        cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction='mean')
        data, target = val_batch

        output = self.model(data)

        logits = F.log_softmax(output, dim=1)
        pred = torch.argmax(logits, dim=1)

        loss = cross_entropy_loss(output, target)

        self.accuracy(pred, target)
        self.log('val_acc_step', self.accuracy, prog_bar=True, logger=True)
        self.log('val_loss', loss, prog_bar=True, logger=True)

        return logits

In [21]:
model_trained = NetPL.load_from_checkpoint('/content/epoch=19-step=1960.ckpt')

In [22]:
model_trained.model

Net(
  (prep): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_bn_pool_1): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res_1): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (res_2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2)

In [67]:
from pytorch_grad_cam.base_cam import ClassifierOutputTarget
import numpy as np
import gradio as gr
from PIL import Image
from pytorch_grad_cam import GradCAM

def gradcam(input_img, transparancy):

    test_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.4913997551666284, 0.48215855929893703, 0.4465309133731618),
                                                             (0.24703225141799082, 0.24348516474564, 0.26158783926049628))
                                        ])

    # target_layers = [model.layer3[-1]]
    input_tensor = test_transforms(input_img.astype(float) / 255.0)

    input_tensor = input_tensor.to(device).float()
    input_tensor = input_tensor.unsqueeze(0)

    output = model_trained.model(input_tensor)

    softmax = F.softmax(output, dim=1)
    classes = list(labels.values())
    probs = softmax.flatten().round(decimals=2).cpu().detach().tolist()
    class_probs = dict(zip(classes, probs))
    print(class_probs)

    logits = F.log_softmax(output, dim=1)
    prediction = torch.argmax(logits, dim=1)

    # Construct the CAM object once, and then re-use it on many images:
    target_layers = [model_trained.model.res_3[-1]]

    cam = GradCAM(model=model_trained, target_layers=target_layers, use_cuda=True)

    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(prediction)])
    grayscale_cam = grayscale_cam[0, :]

    visualization = show_cam_on_image(input_img / 255.0, grayscale_cam, use_rgb=True, image_weight=transparancy)

    return labels[prediction[0].item()], visualization

demo = gr.Interface(gradcam, [gr.Image(shape=(32, 32)), gr.Slider(0, 1)], ["text", gr.Image(shape=(32, 32)).style(width=256, height=256)])
demo.launch(debug=True)

<ipython-input-67-443a296a4c2d>:45: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  demo = gr.Interface(gradcam, [gr.Image(shape=(32, 32)), gr.Slider(0, 1)], ["text", gr.Image(shape=(32, 32)).style(width=256, height=256)])


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

{'airplane': 0.0, 'automobile': 0.0, 'bird': 0.0, 'cat': 0.9900000095367432, 'deer': 0.0, 'dog': 0.009999999776482582, 'frog': 0.0, 'horse': 0.0, 'ship': 0.0, 'truck': 0.0}
Keyboard interruption in main thread... closing server.


In [ ]:
!unzip /content/misclassified.zip

In [49]:
import numpy as np
import gradio as gr

layers = {'Layer - 1': model_trained.model.res_1[-1],
          'Layer - 2': model_trained.model.res_2[-1],
          'Layer - 3': model_trained.model.res_3[-1],
          'Layer - 4': model_trained.model.res_4[-1]}

misclassified_images_paths = glob.glob('/content/content/misclassified/*.jpg')

def gradcam(input_img, transparancy, index_layers, num_classes):

    test_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.4913997551666284, 0.48215855929893703, 0.4465309133731618),
                                                             (0.24703225141799082, 0.24348516474564, 0.26158783926049628))
                                        ])

    # target_layers = [model.layer3[-1]]
    input_tensor = test_transforms(input_img.astype(float) / 255.0)

    input_tensor = input_tensor.to(device).float()
    input_tensor = input_tensor.unsqueeze(0)

    output = model_trained.model(input_tensor)

    softmax = F.softmax(output, dim=1)
    classes = list(labels.values())
    probs = softmax.flatten().cpu().detach().tolist()
    class_probs = dict(zip(classes, probs))
    class_probs = dict(sorted(class_probs.items(), key=lambda x: x[1], reverse=True)[:num_classes])

    logits = F.log_softmax(output, dim=1)
    prediction = torch.argmax(logits, dim=1)

    # Construct the CAM object once, and then re-use it on many images:
    # target_layers = [model_trained.model.res_3[-1]]
    target_layers = [layers[i] for i in index_layers]

    _visualizations = []

    for _layer in index_layers:
        target_layers = [layers[_layer]]

        cam = GradCAM(model=model_trained, target_layers=target_layers, use_cuda=True)

        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(prediction)])
        grayscale_cam = grayscale_cam[0, :]

        visualization = show_cam_on_image(input_img / 255.0, grayscale_cam, use_rgb=True, image_weight=transparancy)

        _visualizations.append(visualization)

    return _visualizations, class_probs

def on_submit(img, transparancy, num_classes, cb_group, num_misclassified_images):
    _vizualizations, class_probs = gradcam(img, transparancy, cb_group, num_classes)
    _misclassified_images = random.sample(misclassified_images_paths, k=int(num_misclassified_images))
    return _vizualizations, _misclassified_images, class_probs

with gr.Blocks() as demo:
    gr.Markdown("CIFAR10 DavidNet")

    with gr.Accordion("Upload Image", open=True):
        img = gr.Image(shape=(32, 32))
        num_classes = gr.Slider(1, 10, step=1.0, value=5.0, label='Number of classes', interactive=True)
        transparancy = gr.Slider(0, 1, value=0.6, label='Transparency', interactive=True)

    with gr.Row(visible=True) as row:
      with gr.Accordion("GradCAM settings", open=True):
          gr.Markdown("This configuration sestion lets users select to GradCAM layers to visialize")
          cb_group = gr.CheckboxGroup(['Layer - 1', 'Layer - 2', 'Layer - 3', 'Layer - 4'],
                                      value=['Layer - 1', 'Layer - 2', 'Layer - 3', 'Layer - 4'],
                                      label="Gradcam Layers",
                                      info="Which all layers should have gradcam",
                                      visible=True)

      with gr.Accordion("Misclassified images", open=True):
          gr.Markdown("This configuration lets users see misclassified images in the result")
          num_misclassified_images = gr.Number(minimum=1.0, maximum=25.0, value=7.0, interactive=True, label='Number of misclassified images to be shown in result')

    example = gr.Examples(
        examples=['/content/content/misclassified/fg_00_actual__cat_predicted__frog.jpg'],
        inputs=[img],
        outputs=[]
    )
    submit = gr.Button("Submit")

    with gr.Column(visible=True) as output_col1:
        gallery = gr.Gallery(label="Gradcam images", show_label=True, elem_id="gradcam", columns=4, show_download_button=True)
        label = gr.Label(label='Predicted classes', num_top_classes=10)

    def on_num_classes_change(num_classes):
        return gr.update(label=f'Predicted class - Top {num_classes} classes')

    misclassified_images = gr.Gallery(label="Misclassified images", show_label=True, elem_id="misclassified", columns=10)
    submit.click(on_submit, inputs=[img, transparancy, num_classes, cb_group, num_misclassified_images], outputs=[gallery, misclassified_images, label])
    num_classes.change(on_num_classes_change, inputs=[num_classes], outputs=[label])

demo.launch(debug=False)

<ipython-input-49-e51dbbe2ba51>:90: GradioUnusedKwargWarning: You have unused kwarg parameters in Gallery, please remove them: {'show_download_button': True}
  gallery = gr.Gallery(label="Gradcam images", show_label=True, elem_id="gradcam", columns=4, show_download_button=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>